In [3]:
!pip install openai

  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached jiter-0.10.0-cp313-cp313-win_amd64.whl.metadata (5.3 kB)
  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.33.2-cp313-cp313-win_amd64.whl.metadata (6.9 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl.metadata (2.6 kB)
   ---------------------------------------- 0.0/767.7 kB ? eta -:--:--
   --------------------------------------- 767.7/767.7 kB 14.6 MB/s eta 0:00:00
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.9-py3-none-any.whl (78 kB)
Using cached jiter-0.10.0-cp313-cp313-win_amd64.whl (205 kB)
Using cached pydantic-2.11.7-py3-none-any.whl (444 kB)
Using cached pydantic_core

In [ ]:
# import openai
# import time
# from dotenv import load_dotenv

# load_dotenv()



# model_name = "gpt-4o"

# # 1. 하이픈 기준 청크 분할 함수
# def split_by_hyphen(text, delim="----------------------------------------"):
#     chunks = [c.strip() for c in text.split(delim) if c.strip()]
#     return chunks

# # 2. 프롬프트 함수 (동일)
# def make_prompt(article_chunk):
#     prompt = f"""
# 너는 한글 자동차 기사에서 QA(질문-답변) 데이터셋을 뽑는 어시스턴트야.
# 아래 기사에서 핵심 내용을 기반으로 중복되지 않는 질문-답변 쌍을 5~10개 생성해줘.
# 각 질문은 한글로, 답변도 기사에 근거해서 한글로 작성해.
# 무조건 아래 기사에서 기반해서 QA 데이터셋을 만들어.
# 출력 예시:
# ```json
# {"messages": [
#   {"role": "user", "content": "엑사원에 대해 알려줘."},
#   {"role": "assistant", "content": "엑사원은 LG AI Research에서 개발한 대규모 언어 모델입니다."},
#   {"role": "user", "content": "개발한 곳이 LG AI Research라고? 정확히 어떤 작업을 할 수 있어?"},
#   {"role": "assistant", "content": "네, 맞습니다. 텍스트 생성, 요약, 번역 등 다양한 자연어 처리 작업을 수행할 수 있습니다. 특히 추론 능력에 강점이 있습니다."},
#   {"role": "user", "content": "그럼 엑사원 파인튜닝은 어떻게 하는지 알려줘."},
#   {"role": "assistant", "content": "엑사원 파인튜닝은 Hugging Face Transformers 라이브러리를 사용하여 LoRA 같은 효율적인 방법을 통해 진행할 수 있습니다. 자체 데이터셋을 준비하여 모델을 특정 도메인에 특화시킬 수 있습니다."}
# ]}
# ```

# 기사:
# {article_chunk}
# """
#     return prompt

# # 3. 반복 QA 추출 함수 (동일)
# def extract_qa_from_chunk(chunk, model_name="gpt-4o", max_tokens=2048, temperature=0.2, retries=3):
#     for i in range(retries):
#         try:
#             response = openai.chat.completions.create(
#                 model=model_name,
#                 messages=[{"role": "user", "content": make_prompt(chunk)}],
#                 max_tokens=max_tokens,
#                 temperature=temperature,
#             )
#             return response.choices[0].message.content
#         except Exception as e:
#             print(f"[오류] 재시도 {i+1}: {e}")
#             time.sleep(2)
#     return "[에러] 추출 실패!"

# # 4. 메인 파이프라인
# def main():
#     # --- txt 파일 읽기 ---
#     with open("text_data/finetuning/new_articles.txt", "r", encoding="utf-8") as f:
#         text = f.read()
#     print(f"총 {len(text)}자 텍스트 로드 완료")

#     # --- 하이픈 기준 청크 분할 ---
#     chunks = split_by_hyphen(text)
#     print(f"총 {len(chunks)}개 chunk로 분할됨!")

#     # --- 반복 QA 추출 및 저장 ---
#     all_qa = []
#     for idx, chunk in enumerate(chunks):
#         print(f"[{idx+1}/{len(chunks)}] QA 추출 중...")
#         qa_text = extract_qa_from_chunk(chunk, model_name=model_name)
#         chat_msgs = qa_text_to_chat_json(qa_text)
#         # 하나의 json 오브젝트에 messages 필드로 저장 (jsonl 한 줄)
#         all_qa_jsonl.append({"messages": chat_msgs})

# 		# 파일로 저장
#         with open("qa_pairs_all.jsonl", "w", encoding="utf-8") as f:
#             for item in all_qa_jsonl:
#                 f.write(json.dumps(item, ensure_ascii=False) + "\n")

#     print("✅ 전체 QA 데이터 저장 완료 → qa_pairs_all.txt")

# if __name__ == "__main__":
#     main()


총 466351자 텍스트 로드 완료
총 187개 chunk로 분할됨!
[1/187] QA 추출 중...


KeyboardInterrupt: 

In [ ]:
import openai
import json
import time
from dotenv import load_dotenv

load_dotenv()

model_name = "gpt-4o"

# 1. 하이픈 기준 청크 분할
def split_by_hyphen(text, delim="----------------------------------------"):
    return [c.strip() for c in text.split(delim) if c.strip()]

# 2. 충실한 답변 생성용 프롬프트
def make_prompt(article_chunk):
    prompt = f"""
너는 자동차 기사로부터 학습용 QA(질문-답변) 데이터셋을 만드는 어시스턴트야.
아래 기사 내용을 바탕으로, 정보가 겹치지 않는 다양한 질문-답변 쌍을 5~10개 생성해줘.
각 질문은 한글로, 답변도 한글로 반드시 기사에 근거해서 작성하되,
답변이 가능한 한 충실하고 자세하게(2문장 이상, 핵심+배경+수치/예시 등 포함) 만들어줘.


출력 형식:
Q: (질문)
A: (답변)

기사:
{article_chunk}
"""
    return prompt

# 3. ChatCompletion(v1.x) 함수
def extract_qa_from_chunk(chunk, model_name="gpt-4o", max_tokens=2048, temperature=0.3, retries=3):
    for i in range(retries):
        try:
            response = openai.chat.completions.create(
                model=model_name,
                messages=[{"role": "user", "content": make_prompt(chunk)}],
                max_tokens=max_tokens,
                temperature=temperature,
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"[오류] 재시도 {i+1}: {e}")
            time.sleep(2)
    return "[에러] 추출 실패!"

# 4. QA 텍스트를 messages 배열로 변환
def qa_text_to_messages(qa_text):
    messages = []
    lines = [line.strip() for line in qa_text.split('\n') if line.strip()]
    last_q = None
    for line in lines:
        if line.startswith("Q:"):
            last_q = line[2:].strip()
            messages.append({"role": "user", "content": last_q})
        elif line.startswith("A:") and last_q is not None:
            messages.append({"role": "assistant", "content": line[2:].strip()})
    return messages

In [13]:

# 5. 메인 파이프라인
def main(file_name):
    with open(f"text_data/finetuning/{file_name}.txt", "r", encoding="utf-8") as f:
        text = f.read()
    print(f"총 {len(text)}자 텍스트 로드 완료")

    # 하이픈 기준 청크
    chunks = split_by_hyphen(text)
    print(f"총 {len(chunks)}개 chunk로 분할됨!")

    # 반복 QA 추출 및 json 저장
    with open(f"text_data/QA_json/{file_name}.json", "w", encoding="utf-8") as f:
        for idx, chunk in enumerate(chunks):
            print(f"[{idx+1}/{len(chunks)}] QA 추출 중...")
            qa_text = extract_qa_from_chunk(chunk, model_name=model_name)
            messages = qa_text_to_messages(qa_text)
            if messages:
                json_obj = {"messages": messages}
                f.write(json.dumps(json_obj, ensure_ascii=False) + "\n")
            time.sleep(1.1)  # API 부하 방지

    print(f"✅ 전체 QA 데이터 저장 완료 → {file_name}.json")

In [12]:
if __name__ == "__main__":
    main("현대 디자인 모토")


총 1054자 텍스트 로드 완료
총 1개 chunk로 분할됨!
[1/1] QA 추출 중...
✅ 전체 QA 데이터 저장 완료 → qa_pairs_all.json
